# Chaincode

This chaincode example is based on the example given at [https://github.com/hyperledger/fabric-samples/tree/master/chaincode/fabcar/java](https://github.com/hyperledger/fabric-samples/tree/master/chaincode/fabcar/java). It's a newer way of building chaincodes in Fabric with Java. In this section we show to construct a simple car repair smart contract. The implementation is divided into two parts: the Car and the chaincode implementation.

## Car implementation

Both classes are placed in the same package. We use the DataType annotation to define the Car as the data type used by our smart contract.

In [ ]:
package com.codete.fabric.examples.oltcar;

import java.util.Objects;

import org.hyperledger.fabric.contract.annotation.DataType;
import org.hyperledger.fabric.contract.annotation.Property;

import com.owlike.genson.annotation.JsonProperty;

@DataType()
public final class Car {


The second part of the ``Car`` class are the properties. In our example we have only two: the mileage and the repairName. Each property has a getter method implemented in the next part of the class. We set the properties using the constructor of the class. Simple.

In [ ]:
    @Property()
    private final Integer mileage;

    @Property()
    private final String repairName;


    public String getMileage() {
        return Integer.toString(mileage);
    }

    public String getRepairName() {
        return repairName;
    }

    public Car(@JsonProperty("mileage") final Integer mileage, @JsonProperty("repairName") final String repairName) {
        this.mileage = mileage;
        this.repairName = repairName;
    }

The next three methods are overriden methods like ``equals`` to compare the objects, ``hashCode`` to generate the hashcode of the object and ``toString`` method to return the property values as string.

In [ ]:
    @Override
    public boolean equals(final Object obj) {
        if (this == obj) {
            return true;
        }

        if ((obj == null) || (getClass() != obj.getClass())) {
            return false;
        }

        Car other = (Car) obj;

        return Objects.deepEquals(new String[] {getMileage(), getRepairName()},
                new String[] {other.getMileage(), other.getRepairName()});
    }

    @Override
    public int hashCode() {
        return Objects.hash(getMileage(), getRepairName());
    }

    @Override
    public String toString() {
        return this.getClass().getSimpleName() + "@" + Integer.toHexString(hashCode())
         + " [mileage=" + Integer.toString(mileage) + ", repairName=" + repairName + "]";
    }

## OLTCar

The smart contract is implemented as the OLTCar class. We need to import more libarires than in the ``Car`` class.

In [ ]:
package com.codete.fabric.examples.oltcar;

import java.util.ArrayList;
import java.util.List;
import java.util.Iterator;

import org.hyperledger.fabric.contract.Context;
import org.hyperledger.fabric.contract.ContractInterface;
import org.hyperledger.fabric.contract.annotation.Contact;
import org.hyperledger.fabric.contract.annotation.Contract;
import org.hyperledger.fabric.contract.annotation.Default;
import org.hyperledger.fabric.contract.annotation.Info;
import org.hyperledger.fabric.contract.annotation.License;
import org.hyperledger.fabric.contract.annotation.Transaction;
import org.hyperledger.fabric.shim.ChaincodeStub;
import org.hyperledger.fabric.shim.ledger.QueryResultsIterator;
import org.hyperledger.fabric.shim.ledger.KeyModification;

import com.owlike.genson.Genson;

The new way of creating a chaincode definition in Fabric can be done with an annotation ``@Contract`` as below.

In [ ]:
@Contract(
        name = "OLTCar",
        info = @Info(
                title = "OLTCar contract",
                description = "OLT example contract",
                version = "0.0.1",
                license = @License(
                        name = "Apache 2.0 License",
                        url = "http://www.apache.org/licenses/LICENSE-2.0.html"),
                contact = @Contact(
                        email = "karol@codete.com",
                        name = "Karol Przystalski",
                        url = "https://www.codete.com")))

To build a smart contract we need to implement ``ContractInterface`` interface. Genson is used to work with JSON data.

In [ ]:
@Default
public final class OLTCar implements ContractInterface {

    private final Genson genson = new Genson();

We implement two transactions ``addCarRepair`` and take the car vin number, repairment name and the mileage of the car when the repair is done. The ``ChaincodeStub`` is the object type that is created to add new block with the method ``putStringState``. We put as a String.

In [ ]:
    /**
     * Creates a new car on the ledger.
     *
     * @param ctx the transaction context
     * @param vin the vin for the new car
     * @param mileage the make of the new car
     * @param repairName the model of the new car
     * @return the created Car entry
     */
    @Transaction()
    public Car addCarRepair(final Context ctx, final String vin, final Integer mileage, final String repairName) {
        ChaincodeStub stub = ctx.getStub();

        Car car = new Car(mileage, repairName);
        String carState = genson.serialize(car);
        stub.putStringState(vin, carState);

        return car;
    }

The car history query method is a bit more complex, because we need to build a list of cars and iterate through the outcome from the blockchain.

In [ ]:
    /**
     * Retrieves every car history.
     *
     * @param ctx the transaction context
     * @return array of Cars events found on the ledger
     */
    @Transaction()
    public Car[] queryCarsHistory(final Context ctx, final String vin) {
        ChaincodeStub stub = ctx.getStub();

        List<Car> cars = new ArrayList<Car>();

        QueryResultsIterator<KeyModification> results = stub.getHistoryForKey(vin);

        Iterator<KeyModification> iter = results.iterator();
        while (iter.hasNext()) {
          Car car = genson.deserialize(iter.next().getStringValue(), Car.class);
          cars.add(car);
        }

        Car[] response = cars.toArray(new Car[cars.size()]);

        return response;
    }

## Build

We build the chaincode using Gradle. We use a typical gradle build configuration.

In [ ]:
plugins {
    id 'checkstyle'
    id 'com.github.johnrengelman.shadow' version '2.0.4'
    id 'java-library'
    id 'jacoco'
}

group 'com.codete.fabric.examples'
version '0.1'

dependencies {
    implementation 'org.hyperledger.fabric-chaincode-java:fabric-chaincode-shim:2.0.0-beta.1'
    implementation 'com.owlike:genson:1.5'
    testImplementation 'org.junit.jupiter:junit-jupiter:5.4.2'
    testImplementation 'org.assertj:assertj-core:3.11.1'
    testImplementation 'org.mockito:mockito-core:2.+'
}

repositories {
    maven {
        url "https://nexus.hyperledger.org/content/repositories/snapshots/"
    }
    jcenter()
    maven {
        url 'https://jitpack.io'
    }
}

checkstyle {
    toolVersion '8.21'
    configFile file("config/checkstyle/checkstyle.xml")
}

checkstyleMain {
    source ='src/main/java'
}

checkstyleTest {
    source ='src/test/java'
}

shadowJar {
    baseName = 'chaincode'
    version = null
    classifier = null
    manifest {
        attributes 'Main-Class': 'org.hyperledger.fabric.contract.ContractRouter'
    }
}

test {
    useJUnitPlatform()
    testLogging {
        events "passed", "skipped", "failed"
    }
}

To get the bash console in our ``fabric_cli`` container, you can use the following command: 

``docker exec -it fabric_cli bash``

Then you can go to the directory where the source code is placed (``/home/codete/workshop/``) and execute the gradle build method:

``./gradlew clean build shadowJar``

# Deploy 

To deploy the chaincode, you should use the ``install`` command and ``instantinate`` as following:

``peer chaincode install -l java -n carcc -v v1 -p <path>``

``peer chaincode instantiate -o orderer.codete.com:7050 -C channelone -n carcc -v v1 -c '{"Args":[]}' -P 'OR ("Org1MSP.member")'``

Please remember to change the ``<path>`` to the directory where code witht the build is placed.

# Run

We have implemented two transactions: ``addCarRepair`` and ``queryCarsHistory``. To get the history we should add something to our chain:

``peer chaincode invoke -n carcc -c '{"Args":["addCarRepair", "1234567", 3000, "Wheel change"]}' -C channelone``


The transaction should return the added block. In the next step we can add another block or check the history like:
``peer chaincode invoke -n carcc -c '{"Args":["queryCarsHistory", "1234567"]}' -C channelone``

This method should return the current history of the car repair based on the car vin number.